In [ ]:
#### IMPORTS ####
using ProgressMeter
using Plots, Measures
using LaTeXStrings
using NPZ
theme(:ggplot2)
include("dWaveRgFlow.jl")

In [ ]:
map1DTo2D(points, num_kspace) = [(points .- 1) .% num_kspace .+ 1, div.((points .- 1), num_kspace) + 1]

num_kspace_half = 10
num_kspace = 2 * num_kspace_half + 1
kx_arr = range(-pi, stop=pi, length=num_kspace)
for W_val in [0.004, 0.008, 0.01, 0.02, 0.05, 0.055, 0.06]
    @time kondoJArray, dispersionArray = main(num_kspace_half, 1, 0.05, -W_val)
    FermiArmSouthWest = [num_kspace_half + 1 + j * (num_kspace - 1) for j in 0:(num_kspace_half)]
    FermiArmAbove = [point - 1 for point in FermiArmSouthWest if 1 <= (point - 1) % num_kspace <= num_kspace_half]
    FermiArmBelow = [point + 1 for point in FermiArmSouthWest if 1 <= point % num_kspace <= num_kspace_half]
    
    antinodeD = FermiArmSouthWest[1]
    midway = FermiArmSouthWest[div(1 + num_kspace_half, 4)]
    node = FermiArmSouthWest[div(1 + num_kspace_half, 2)]
    J_antinodeD = zeros(length(FermiArmAbove), num_kspace_half+1)
    J_midway = zeros(length(FermiArmAbove), num_kspace_half+1)
    J_node = zeros(length(FermiArmAbove), num_kspace_half+1)
    for i in 1:(num_kspace_half+1)
        J_antinodeD[:,i] = kondoJArray[antinodeD,FermiArmAbove,i]
        J_midway[:,i] = kondoJArray[midway,FermiArmAbove,i]
        J_node[:,i] = kondoJArray[node,FermiArmAbove,i]
    end
    
    p1 = heatmap(1:(num_kspace_half+1), 1:length(FermiArmAbove), J_node, title="\$\\vec k_1 = (0,-\\pi), W=$W_val\$", c=:thermal)
    p2 = heatmap(1:(num_kspace_half+1), 1:length(FermiArmAbove), J_midway, title="\$\\vec k_1 = (-\\pi/4,-3\\pi/4), W=$W_val\$", c=:thermal)
    p3 = heatmap(1:(num_kspace_half+1), 1:length(FermiArmAbove), J_antinodeD, title="\$\\vec k_1 = (-\\pi/2,-\\pi/2), W=$W_val\$", c=:thermal)
    p= plot(p1, p2, p3, size=(400, 200 * 3), layout=(3, 1),
    titlefontsize=10, topmargin=-2mm, bottommargin=-2mm, leftmargin=3mm, rightmargin=3mm)
    savefig(p, "figCombined_$W_val.pdf")
    p1 = heatmap(1:(num_kspace_half+1), 1:length(FermiArmAbove), log10.(abs.(J_node)), title="\$\\vec k_1 = (0,-\\pi), W=$W_val\$", c=:thermal)
    p2 = heatmap(1:(num_kspace_half+1), 1:length(FermiArmAbove), log10.(abs.(J_midway)), title="\$\\vec k_1 = (-\\pi/4,-3\\pi/4), W=$W_val\$", c=:thermal)
    p3 = heatmap(1:(num_kspace_half+1), 1:length(FermiArmAbove), log10.(abs.(J_antinodeD)), title="\$\\vec k_1 = (-\\pi/2,-\\pi/2), W=$W_val\$", c=:thermal)
    p= plot(p1, p2, p3, size=(400, 200 * 3), layout=(3, 1),
    titlefontsize=10, topmargin=-2mm, bottommargin=-2mm, leftmargin=3mm, rightmargin=3mm)
    savefig(p, "figCombined_abs_$W_val.pdf")
end